作業 : (Kaggle)鐵達尼生存預測  
[作業目標]  
試著完成三種不同特徵類型的三種資料操作, 觀察結果  
思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理  
[作業重點]  
完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])  
思考何種特徵類型, 應該最複雜  

In [1]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [2]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 秀出資料欄位的類型與數量
# df.dtypes : 轉成以欄位為 index, 類別(type)為 value 的 DataFrame
# .reset_index() : 預設是將原本的 index 轉成一個新的欄位, 如果不須保留 index, 則通常會寫成 .reset_index(drop=True)
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [4]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [43]:
#---Integer features
int_f='SibSp'
print("Integer features: %s" %int_f)
print(df[int_f].head())
print('mean %.3f, max %d, unique %s\n' %(df[int_f].mean(), df[int_f].max(), df[int_f].unique()))

#---Float features
float_f='Age'
print("Float features: %s" %float_f)
print(df[float_f].head())
print('mean %.3f, max %d, unique %s\n' %(df[float_f].mean(), df[float_f].max(), df[float_f].unique()))

#---Object features
object_f='Cabin'
print("Object features: %s" %object_f)
print(df[object_f].head())
print('unique %s' %df[object_f].unique() )

Integer features: SibSp
0    1
1    1
2    0
3    1
4    0
Name: SibSp, dtype: int64
mean 0.499, max 8, unique [1 0 3 4 2 5 8]

Float features: Age
0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
Name: Age, dtype: float64
mean 29.881, max 80, unique [22.   38.   26.   35.     nan 54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.   28.    8.   19.   40.   66.   42.   21.
 18.    3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.
 16.   25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.
 14.5  70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.
 56.   50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.
 60.   10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.
  0.67 30.5   0.42 34.5  74.   22.5  18.5  67.   76.   26.5  60.5  11.5
  0.33  0.17 38.5 ]

Object features: Cabin
0     NaN
1     C85
2     NaN
3    C123
4     NaN
Name: Cabin, dtype: object
unique [nan 'C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23

作業1  
試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  
int, float > 操作上沒問題(不會發生error), 不過依照features不同會有不同解讀   
object > 資料型態問題, 平均和最大值函式, 會因為無法計算出現error 

作業2  
思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?   
我認為時間型特徵處理上最複雜；數值型特徵可以透過數值運算看出平均值、最大最小值、眾數等資訊；類別型資料可以看出種類個數、出現頻率等；  
在函式處理上較為簡單，可以用函式或是簡單的迴圈計算，也較為直觀；時間型的特徵，則需要轉換成天、小時或分鐘，來比較差異。  